In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from mqt.predictor import rl, ml
df_backup = pd.read_csv(ml.helper.get_path_trained_model() / 'backup_230828_res.csv', sep=',')
df_backup = df_backup.reset_index()
df = pd.read_csv(ml.helper.get_path_trained_model() / 'res.csv', sep=',')

In [ ]:
columns_to_join = [col for col in df_backup.columns if "MQT" not in col and col not in df.columns]

df = df.join(df_backup[columns_to_join])

In [ ]:
df["benchmark_id"]=df["benchmark"].astype(str)+ "_" + df["num_qubits"].astype(str)+ "_" + df["index"].astype(str)

## Add best datapoints from other compilation flows

In [ ]:
## Add third highest data point, mean, best and worst

In [ ]:
def calculate_x_highest(row, column_name, index):
    columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
    values = sorted([value for col, value in row[columns].items() if value != -1], reverse=True)
    return values[index-1] if len(values) >= 3 else None

def calculate_mean(row, column_name):
    columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
    values = [value for col, value in row[columns].items() if value != -1]
    return np.mean(values) if values else None

df['fidelity_second_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=2, axis=1)
df['fidelity_third_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=3, axis=1)
df['fidelity_mean'] = df.apply(calculate_mean, column_name="_fidelity",axis=1)


df['critical_depth_third_highest'] = df.apply(calculate_x_highest, column_name="_critical_depth", index=3,axis=1)
df['critical_depth_mean'] = df.apply(calculate_mean, column_name="_critical_depth",axis=1)

In [ ]:
df['fidelity_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['fidelity_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].max(), axis=1)
df['critical_depth_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['critical_depth_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].max(), axis=1)


In [ ]:
predictor = ml.Predictor()
(X_train,
X_test,
y_train,
y_test,
indices_train,
indices_test,
names_list,
scores_list) = predictor.get_prepared_training_data(figure_of_merit="fidelity", save_non_zero_indices=True)
test_benchmarks_fidelity = [names_list[index_test] for index_test in indices_test]
df_filtered_fidelity = df[df["file_path"].isin(test_benchmarks_fidelity)]

In [ ]:
plot_kind = "line"
df = df_filtered_fidelity.sort_values(by=['MQTPredictor_fidelity_fidelity'])
ax = df.plot(x="benchmark_id", y="fidelity_max_other", label="max_other", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df.plot(x="benchmark_id", y="fidelity_min_other", label="min_other", rot=90, kind=plot_kind, ax=ax, color="red")
df.plot(x="benchmark_id", y="MQTPredictor_fidelity_fidelity", kind=plot_kind, rot=90, ax=ax, color="blue")
df.plot(x="benchmark_id", y="fidelity_third_highest", kind=plot_kind, rot=90, ax=ax, color="purple")
df.plot(x="benchmark_id", y="fidelity_second_highest", kind=plot_kind, rot=90, ax=ax, color="black")
df.plot(x="benchmark_id", y="fidelity_mean", kind=plot_kind, rot=90, ax=ax, color="orange")

if plot_kind == "line":
    plt.xticks(range(len(df.benchmark_id)), df.benchmark_id)
plt.savefig("results/fidelity_plot.pdf", bbox_inches='tight')

In [ ]:
predictor = ml.Predictor()
(X_train,
X_test,
y_train,
y_test,
indices_train,
indices_test,
names_list,
scores_list) = predictor.get_prepared_training_data(figure_of_merit="critical_depth", save_non_zero_indices=True)
test_benchmarks_critical_depth = [names_list[index_test] for index_test in indices_test]
df_filtered_critical_depth = df[df["file_path"].isin(test_benchmarks_critical_depth)]

In [ ]:
plot_kind = "scatter"
df = df_filtered_critical_depth.sort_values(by=['MQTPredictor_critical_depth_critical_depth'])
ax = df.plot(x="benchmark_id", y="critical_depth_max_other", label="max_other", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df.plot(x="benchmark_id", y="critical_depth_min_other", label="min_other", rot=90, kind=plot_kind, ax=ax, color="red")
df.plot(x="benchmark_id", y="MQTPredictor_critical_depth_critical_depth", kind=plot_kind, rot=90, ax=ax, color="blue")
df.plot(x="benchmark_id", y="critical_depth_third_highest", kind=plot_kind, rot=90, ax=ax, color="black")
df.plot(x="benchmark_id", y="critical_depth_mean", kind=plot_kind, rot=90, ax=ax, color="orange")

if plot_kind == "line":
    plt.xticks(range(len(df.benchmark_id)), df.benchmark_id);
plt.savefig("results/critical_depth_plot.pdf", bbox_inches='tight')

## Normalize

In [ ]:
# for index, row in df.iterrows():
#     max_fidelity = max(row[col] for col in df.columns if col.endswith('_fidelity') and "MQT" not in col)
#     for col in df.columns:
#         if col.endswith('_fidelity'):
#             df.at[index, col] /= max_fidelity

# for index, row in df.iterrows():
#     max_crit_depth = max(row[col] for col in df.columns if col.endswith('_critical_depth') and "MQT" not in col)
#     if max_crit_depth == 0:
#         continue
#     for col in df.columns:
#         if col.endswith('_critical_depth'):
#             df.at[index, col] /= max_crit_depth

# New Evaluation Scheme

In [ ]:
list_cols_fid = [x for x in df.columns if x.endswith("fidelity")]

In [ ]:
cmap = plt.get_cmap("viridis", len(list_cols_fid))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]

# New Rankings

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xlim(bins[0], bins[-1])

In [ ]:
df['Rank_MQT_fidelity'] = df.apply(lambda row: sum(1 for col in df.columns if col.endswith('_fidelity') and not "MQT" in col and row['MQTPredictor_fidelity_fidelity'] >= row[col]), axis=1)
df['Rank_MQT_fidelity'] = len([col for col in df.columns if col.endswith('_fidelity') and not "MQT" in col]) - df.Rank_MQT_fidelity
plt.hist(df.Rank_MQT_fidelity.values, bins=range(0,15,1), align="left")
plt.xticks(range(0,14,1));

In [ ]:
len(df[df['Rank_MQT_fidelity']==0])

In [ ]:
df['Rank_MQT_critical_depth'] = df.apply(lambda row: sum(1 for col in df.columns if col.endswith('_critical_depth') and not "MQT" in col and row['MQTPredictor_critical_depth_critical_depth'] >= row[col]), axis=1)
df['Rank_MQT_critical_depth'] = len([col for col in df.columns if col.endswith('_critical_depth') and not "MQT" in col]) - df.Rank_MQT_critical_depth
plt.hist(df.Rank_MQT_critical_depth.values, bins=range(0,15,1), align="left")
plt.xticks(range(0,14,1));

In [ ]:
len(df[df['Rank_MQT_critical_depth']==0])

# Old

In [ ]:
df = pd.read_csv(ml.helper.get_path_trained_model() / 'backup_230828_res.csv', sep=',')

In [ ]:
df.MQTPredictor_fidelity_critical_depth.describe()

In [ ]:
df.qiskit_ibm_washington_critical_depth.describe()

In [ ]:
df.tket_ibm_washington_critical_depth.describe()

In [ ]:
num_bins = 60
color_qiskit = "#440154"
color_TKET = "#21918c"
size = 14
legendsize = 12

In [ ]:
df["MQTPredictor-Qiskit_Fid"]=df["MQTPredictor_fidelity_fidelity"]-df["qiskit_ibm_washington_fidelity"]
df["MQTPredictor-TKET_Fid"]=df["MQTPredictor_fidelity_fidelity"]-df["tket_ibm_washington_fidelity"]

In [ ]:
df["MQTPredictor-Qiskit_Fid"][df["MQTPredictor-Qiskit_Fid"] > 0.0 ].count()/len(df)

In [ ]:
df["MQTPredictor-TKET_Fid"][df["MQTPredictor-TKET_Fid"] > 0.0 ].count()/len(df)

In [ ]:
df["MQTPredictor-Qiskit_Fid"]=df["MQTPredictor_fidelity_fidelity"]-df["qiskit_ibm_washington_fidelity"]
df["MQTPredictor-tket_ibm_washington_Fid"]=df["MQTPredictor_fidelity_fidelity"]-df["tket_ibm_washington_fidelity"]
df["MQTPredictor-Qiskit_Crit_Depth"]=df["MQTPredictor_critical_depth_critical_depth"]-df["qiskit_ibm_washington_critical_depth"]
df["MQTPredictor-TKET_Crit_Depth"]=df["MQTPredictor_critical_depth_critical_depth"]-df["tket_ibm_washington_critical_depth"]

In [ ]:

max_val_bench = max([df.groupby(["benchmark"])["MQTPredictor-Qiskit_Fid"].mean().max(), 
                     df.groupby(["benchmark"])["MQTPredictor-TKET_Fid"].mean().max(), 
                     df.groupby(["benchmark"])["MQTPredictor-Qiskit_Crit_Depth"].mean().max(), 
                     df.groupby(["benchmark"])["MQTPredictor-TKET_Crit_Depth"].mean().max()
                    ]
                     )
min_value_bench = min([df.groupby(["benchmark"])["MQTPredictor-Qiskit_Fid"].mean().min(),
                      df.groupby(["benchmark"])["MQTPredictor-TKET_Fid"].mean().min(), 
                      df.groupby(["benchmark"])["MQTPredictor-Qiskit_Crit_Depth"].mean().min(), 
                      df.groupby(["benchmark"])["MQTPredictor-TKET_Crit_Depth"].mean().min()])

In [ ]:
df["MQTPredictor-Qiskit_Fid"][df["MQTPredictor-Qiskit_Fid"] > 0.0 ].count()/len(df)

In [ ]:
df["MQTPredictor-TKET_Fid"][df["MQTPredictor-TKET_Fid"] > 0.0 ].count()/len(df)

In [ ]:
df["MQTPredictor-Qiskit_Crit_Depth"][df["MQTPredictor-Qiskit_Crit_Depth"] >= 0.0 ].count()/len(df)

In [ ]:
df["MQTPredictor-TKET_Crit_Depth"][df["MQTPredictor-TKET_Crit_Depth"] >= 0.0 ].count()/len(df)

# Expected Fidelity

In [ ]:
max_val = max([df["MQTPredictor-Qiskit_Fid"].max(), df["MQTPredictor-TKET_Fid"].max(), -df["MQTPredictor-Qiskit_Fid"].min(), -df["MQTPredictor-TKET_Fid"].min()])
xrange = (-max_val-(max_val)/10, max_val+(max_val)/10)
ax = df["MQTPredictor-Qiskit_Fid"].hist(bins=num_bins, alpha=0.5, range=xrange, color=color_qiskit, weights=np.ones(len(df["MQTPredictor-Qiskit_Fid"])) / len(df["MQTPredictor-Qiskit_Fid"]), xlabelsize=size, ylabelsize=size)
df["MQTPredictor-TKET_Fid"].hist(bins=num_bins, ax=ax, alpha=0.5, range=xrange, color=color_TKET, grid=False, weights=np.ones(len(df["MQTPredictor-TKET_Fid"])) / len(df["MQTPredictor-TKET_Fid"]))
ax.set_ylabel("Relative frequency", size=size)
ax.set_xlabel("Absolute reward  difference", size=size)
ax.legend(["compared to Qiskit", "compared to TKET"], fontsize=legendsize, loc='upper right')
ax.axvline(x=0, color= 'black', linewidth=1).get_figure().savefig('results/fidelity_hist.pdf', bbox_inches='tight')

In [ ]:
ax = df.groupby(["benchmark"])["MQTPredictor-Qiskit_Fid"].mean().plot(kind="bar", color=color_qiskit, alpha=0.5)
df.groupby(["benchmark"])["MQTPredictor-TKET_Fid"].mean().plot(kind="bar", ax=ax, color=color_TKET, alpha=0.5)
ax.set_ylabel("Average reward difference", size=size)
ax.set_xlabel("Benchmark", size=size)
ax.tick_params(axis='x', which='both', labelsize=size)
ax.legend(["compared to Qiskit", "compared to TKET"], fontsize=legendsize, loc='upper right')
#ax.set_ylim(min_value_bench-0.01, max_val_bench+0.01)
ax.tick_params(axis='both', labelsize=size)
ax.axhline(y=0, color= 'black', linewidth=1).get_figure().savefig('results/fidelity_bench.pdf', bbox_inches='tight')

In [ ]:
print(df["MQTPredictor-Qiskit_Fid"].mean(), df["MQTPredictor-TKET_Fid"].mean())

# Critical Depth

In [ ]:
max_val = max([df["MQTPredictor-Qiskit_Crit_Depth"].max(), df["MQTPredictor-TKET_Crit_Depth"].max(), -df["MQTPredictor-Qiskit_Crit_Depth"].min(), -df["MQTPredictor-TKET_Crit_Depth"].min()])
xrange = (-max_val-(max_val)/10, max_val+(max_val)/10)
ax = df["MQTPredictor-Qiskit_Crit_Depth"].hist(bins=num_bins, alpha=0.5, range=xrange, color=color_qiskit, weights=np.ones(len(df["MQTPredictor-Qiskit_Crit_Depth"])) / len(df["MQTPredictor-Qiskit_Crit_Depth"]), xlabelsize=size, ylabelsize=size)
df["MQTPredictor-TKET_Crit_Depth"].hist(bins=num_bins, ax=ax, alpha=0.5, range=xrange, grid=False, color=color_TKET, weights=np.ones(len(df["MQTPredictor-TKET_Crit_Depth"])) / len(df["MQTPredictor-TKET_Crit_Depth"]))
ax.set_ylabel("Relative frequency", size=size)
ax.set_xlabel("Absolute reward difference", size=size)
ax.legend(["compared to Qiskit", "compared to TKET"], fontsize=legendsize, loc='upper right')
ax.axvline(x=0, color= 'black', linewidth=1).get_figure().savefig('results/critical_depth_hist.pdf', bbox_inches='tight')

In [ ]:
ax = df.groupby(["benchmark"])["MQTPredictor-Qiskit_Crit_Depth"].mean().plot(kind="bar", color=color_qiskit, alpha=0.5)
df.groupby(["benchmark"])["MQTPredictor-TKET_Crit_Depth"].mean().plot(kind="bar", ax=ax, color=color_TKET, alpha=0.5)
ax.set_ylabel("Average reward difference", size=size)
ax.set_xlabel("Benchmark", size=size)
ax.tick_params(axis='x', which='both', labelsize=size)
ax.set_ylim(min_value_bench-0.01, max_val_bench+0.01)
ax.tick_params(axis='both', labelsize=size)
ax.legend(["compared to Qiskit", "compared to TKET"], fontsize=legendsize, loc='upper right')
ax.axhline(y=0, color= 'black', linewidth=1).get_figure().savefig('results/critical_depth_bench.pdf', bbox_inches='tight')

In [ ]:
print(df["MQTPredictor-Qiskit_Crit_Depth"].mean(), df["MQTPredictor-TKET_Crit_Depth"].mean())

# 2x2 Matrix With Mean Results and Optimization Criterion Comparison

In [ ]:
MQT_fidelity = [df["MQTPredictor_fidelity_fidelity"].mean(),  df["MQTPredictor_fidelity_critical_depth"].mean()]
MQT_critical_depth = [df["MQTPredictor_critical_depth_fidelity"].mean(), df["MQTPredictor_critical_depth_critical_depth"].mean()]

In [ ]:
print(np.round(MQT_fidelity,2))
print(np.round(MQT_critical_depth,2))